<a href="https://colab.research.google.com/github/humlaye/AI-detection-benchmark/blob/main/Essay_JSON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Bu kod csv dosyasında bulunan metinleri ID'leri ile tek tek bir JSON yapısına getirmektedir.

In [ ]:
import pandas as pd
import json
import os
from google.colab import drive

print("Google Drive bağlanıyor...")
drive.mount('/content/drive')
print("Drive başarıyla bağlandı.")

In [ ]:
#Dosya yolları belirtme yeri
DRIVE_ROOT_PATH = '/content/drive/MyDrive/Fırat Projeler/Detecting AI Influence in Student Writing: Toward Reliable and Interpretable Classifiers/feedback-prize-english-language-learning'
TRAIN_CSV_PATH = os.path.join(DRIVE_ROOT_PATH, 'train.csv')
TEXT_FOLDER = os.path.join(DRIVE_ROOT_PATH, 'train')
OUTPUT_FILE_PATH = os.path.join(DRIVE_ROOT_PATH, 'ell_essay_families_structure.jsonl')

if not os.path.exists(TRAIN_CSV_PATH):
    print(f"\nHATA: '{TRAIN_CSV_PATH}' yolu bulunamadı. Lütfen DRIVE_ROOT_PATH'i kontrol edin.")
else:


    # train.csv dosyasını okuma (Metin ID'leri için)
    print(f"\n'{TRAIN_CSV_PATH}' dosyası okunuyor...")
    df_train = pd.read_csv(TRAIN_CSV_PATH)

    # Sadece metin ID'leri ve skor
    unique_essays = df_train.drop_duplicates(subset=['text_id']).set_index('text_id')

    # JSON yapısını tutacak ana liste
    text_family_data = []

    print(f"\n{len(unique_essays)} adet deneme metni için JSON yapısı oluşturuluyor.")

    # Her bir deneme metnini JSON yapısına dönüştürme
    for text_id in unique_essays.index:
        essay_file_name = f"{text_id}.txt"
        essay_file_path = os.path.join(TEXT_FOLDER, essay_file_name)

        try:
            with open(essay_file_path, 'r', encoding='utf-8') as f:
                original_text = f.read()
        except FileNotFoundError:
            # Hata oluşursa atla
            # print(f"Hata: {essay_file_name} bulunamadı, atlanıyor.")
            continue

        scores = unique_essays.loc[text_id, ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']].to_dict()

        # JSON yapısı
        family_object = {
            "family_id": text_id,
            "source": {
                "type": "student_essay_ell",
                "original_scores": scores,
                "path": essay_file_name
            },
            "versions": [ # İleride buraya AI versiyonların eklenecek.
                {
                    "type": "original",
                    "text": original_text,
                    "length_chars": len(original_text),

                }
            ],
            "detection_results": {},
            "metadata": {
                "creation_date": pd.Timestamp.now().isoformat(),
                "notes": "Original text extracted from Feedback Prize ELL dataset."
            }
        }

        text_family_data.append(family_object)

In [ ]:
#kaydetme
print(f"\nOluşturulan veri {OUTPUT_FILE_PATH} dosyasına kaydediliyor.")

    # JSON Lines formatında kaydetme
    with open(OUTPUT_FILE_PATH, 'w', encoding='utf-8') as f:
        for item in text_family_data:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')

    print("İşlem Tamamlandı")
    print(f"Toplam {len(text_family_data)} adet metin ailesi yapısı oluşturuldu.")
    print(f"JSONL Dosyanız Drive'da kaydedildi: {OUTPUT_FILE_PATH}")


In [ ]:
# var olan bir kısmı kontrol etme
print("\n--- Örnek Yapı (İlk Metin) ---")
print(json.dumps(text_family_data[0], indent=2, ensure_ascii=False))